# Entrenando word vectors desde un corpus plano

In [22]:
import pandas as pd
import re
import torch

In [18]:
with open('hp1.txt','r') as file:
    corpus = file.read()
    corpus = re.sub(r'\n+\d+\n+\x0c',r'\n',corpus)
    corpus = re.sub(r'\n+\d+\n+\b',r'\n\n',corpus)
    corpus = re.sub(r'[^\.:?!]\n+',corpus) # ???
    with open('hp1_v2.txt','w') as file2:
        file2.write(corpus)

<re.Match object; span=(0, 2), match='\n\n'>


Obtengo el corpus con el que se va a trabajar. El mismo consiste en una lista de documentos (strings) que no están separados en tokens.

In [ ]:
def tokenizer(token_series):
    return token_series.split(' ')

corpus = ['El niño que vivió','El señor y la señora Dursley, que vivían en el número 4 de Privet Drive, estaban orgullosos de decir que eran muy normales, afortunadamente. Eran las últimas personas que se esperaría encontrar relacionadas con algo extraño o misterioso, porque no estaban para tales tonterías.','El señor Dursley era el director de una empresa llamada Grunnings, que fabricaba taladros. Era un hombre corpulento y rollizo, casi sin cuello, aunque con un bigote inmenso. La señora Dursley era delgada, rubia y tenía un cuello casi el doble de largo de lo habitual, lo que le resultaba muy útil, ya que pasaba la mayor parte del tiempo estirándolo por encima de la valla de los jardines para espiar a sus vecinos. Los Dursley tenían un hijo pequeño llamado Dudley, y para ellos no había un niño mejor que él.']
corpus = [tokenizer(par) for par in corpus]

Obtengo el vocabulario

In [21]:
from NLPDataUtils import Vocabulary

vocabulary = Vocabulary()
for doc in corpus:
    for token in doc:
        vocabulary.add_token(token)

Defino las muestras

In [43]:
window_size = 2
words = pd.Series([token for doc in corpus for token in doc])
contexts = pd.Series([doc[max(0,i-window_size):i]+doc[i+1:min(i+window_size+1, len(doc))] for doc in corpus for i in range(len(doc))])

Ahora hago todo eso pero adentro de la clase Dataset:

In [76]:
from torch.utils.data import Dataset
import torch.nn as nn


class Word2VecSamples(Dataset):
    
    no_token = '<NT>'
    
    def __init__(self, corpus, window_size=2):
        
        # Obtengo el vocabulario a partir del corpus ya tokenizado:
        self.corpus = corpus
        self.vocabulary = Vocabulary()
        for doc in corpus:
            for token in doc:
                self.vocabulary.add_token(token)
                
        # Obtengo el contexto a partir del corpus:
        self.window_size = window_size
        self.data = pd.DataFrame({'word': [token for doc in corpus for token in doc],
                                  'context': [[self.no_token for j in range(i-window_size, max(0,i-window_size))] + \
                                              doc[max(0,i-window_size):i] + \
                                              doc[i+1:min(i+window_size+1, len(doc))] + \
                                              [self.no_token for j in range(min(i+window_size+1, len(doc)),i+window_size+1)] \
                                              for doc in corpus for i in range(len(doc))]
                                 })
        self.padding_idx = len(self.vocabulary)
        
    def __getitem__(self,idx):
        if type(idx) == torch.Tensor:
            idx = idx.item()
        
        word_vector = torch.tensor(self.vocabulary.token_to_index(self.data['word'].iloc[idx]), dtype=torch.long)
        context_vector = torch.zeros(2 * self.window_size, dtype=torch.long)
        for i, token in enumerate(self.data['context'].iloc[idx]):
            if token == self.no_token:
                context_vector[i] = self.padding_idx
            else:
                context_vector[i] = self.vocabulary.token_to_index(token)
            
        return word_vector, context_vector        
        
        
class CBOWModel(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim):
        super(CBOWModel,self).__init__()
        self.emb = nn.Embedding(vocab_size, embedding_dim, padding_idx=vocab_size)
        self.out = nn.Linear(embedding_dim, vocab_size)
        
    def forward(self,x):
        return self.out(self.emb(x))
    
    def loss(self,scores,target):
        lf = nn.CrossEntropyLoss()
        return lf(scores,target)
        
class SkipGramModel(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim):
        super(SkipGramModel,self).__init__()
        self.emb = nn.Embedding(vocab_size+1, embedding_dim, padding_idx=vocab_size)
        self.out = nn.Linear(embedding_dim, vocab_size)
        self.vocab_size = vocab_size
        
    def forward(self,x):
        return self.out(self.emb(x))
    
    def loss(self,scores,target):
        lf = nn.CrossEntropyLoss()
        if target.size() != torch.Size([2]):
            context_size = target.size(1)
            scores = scores.view(-1,self.vocab_size,1).repeat(1,1,context_size)
            print(scores)
            print(target)
        return lf(scores,target)

SkipGramModel(
  (emb): Embedding(101, 100, padding_idx=100)
  (out): Linear(in_features=100, out_features=100, bias=True)
)

Defino el modelo

In [78]:
from TorchDataUtils import *

def tokenizer(token_series):
    return token_series.split(' ')


# FALTA HACER UNA FUNCIÓN TrainWord2Vec(..., model='SkipGram') QUE NO SIGA EL FORMATO DE UN PROBLEMA
# DE CLASIFICACIÓN COMÚN.
corpus = ['El niño que vivió','El señor y la señora Dursley, que vivían en el número 4 de Privet Drive, estaban orgullosos de decir que eran muy normales, afortunadamente. Eran las últimas personas que se esperaría encontrar relacionadas con algo extraño o misterioso, porque no estaban para tales tonterías.','El señor Dursley era el director de una empresa llamada Grunnings, que fabricaba taladros. Era un hombre corpulento y rollizo, casi sin cuello, aunque con un bigote inmenso. La señora Dursley era delgada, rubia y tenía un cuello casi el doble de largo de lo habitual, lo que le resultaba muy útil, ya que pasaba la mayor parte del tiempo estirándolo por encima de la valla de los jardines para espiar a sus vecinos. Los Dursley tenían un hijo pequeño llamado Dudley, y para ellos no había un niño mejor que él.']
corpus = [tokenizer(par) for par in corpus]
dataset = Word2VecSamples(corpus, window_size=2)
vocab_size = len(samples.vocabulary)
embedding_dim = 100
model = SkipGramModel(vocab_size, embedding_dim)

NameError: name 'train_dataset' is not defined